## Global Pairwise Sequence Alignment 

    Given two sequnce alignment, generate a 3-tuples datastructure of alignment result.
    output: (identity_total, residue_total, alignment result list)
    
    * alignment result list: each element is a 2-tuples node (hk1_api, hk2_api)
      hk1_api format -> (Ordinal number, api+par+return)
      e.g., each element (hk1_api, hk2_api)
      -> ((1, 'LoadLibrary#PR#SYS@imm32@DLL#PR#SUCCESS'), (1, 'LoadLibrary#PR#SYS@imm32@DLL#PR#SUCCESS'))

In [ ]:
#global alignment (Needleman - Wunsch algorithm) 
def seqMatch(h1_list, i, h2_list, j, window):
    for w in range(0, window):
        api = h1_list[i+w]
        a = h2_list[j+w]
        if api != a:
            return 0
    return 1

def pairwise_NW(h1_list ,h2_list, dp_matched, dp_mismatched, dp_gap, isret, offset=0, window=1):
    #isret == 1 add return to list else not add
    h1_list = [(t,a[:a.rindex('#')+1]) for t,a in h1_list] if not isret else h1_list
    h2_list = [(t,a[:a.rindex('#')+1]) for t,a in h2_list] if not isret else h2_list

    isH1Long = True if len(h1_list) >= len(h2_list) else False 
    long_list = h1_list if isH1Long else h2_list
    short_list = h2_list if isH1Long else h1_list

    # calc: dynamic programming, Needleman-Wunsch algorithm, use long_list
    # ,short_list to find best local alignment # MIKE:20141023
    m = len(short_list)
    n = len(long_list)
    
    # MIKE: score matrix, all zero, dynamic programming needs one more row and col
    # NEWMAN: NW score matrix, initialization
    score = [[dp_gap*i if j==0 else 0 for j in range(n+1)] for i in range(m+1)]
    score[0] = [0+dp_gap*j for j in range(n+1)]
    
    # calc: for match 
    matched = [[0 for j in range(n+1)] for i in range(m+1)]
    for i in range(len(short_list)):
        if offset + i + window > len(short_list): break
        for j in range(len(long_list)):
            if offset + j + window > len(long_list): break
            if seqMatch(short_list, offset+i, long_list, offset+j, window):
                matched[i+1][j+1] = 1

    # dynamic programming
    highest_score = 0
    highest_xy = (0,0)
    walk = [[0 for j in range(n+1)] for i in range(m+1)]
    
    #start from 1, the first additional col/row are all zero, no need to process
    for i in range(1, m+1): 
        for j in range(1, n+1):
            # case 1: matched or mismatched
            sc_ab = 0
            if matched[i][j] == 1:
                sc_ab = score[i-1][j-1] + dp_matched
            else:
                sc_ab = score[i-1][j-1] + dp_mismatched

            # case 2: gap y
            max_y = -100000             #===== 0->-10000
            for wy in range(1,j): # 1~j-1
                test = score[i][wy] + (j-wy) * dp_gap
                max_y = test if test > max_y else max_y

            # case 3: gap x
            max_x = -100000             #===== 0->-10000
            for wx in range(1,i):
                test = score[wx][j] + (i-wx) * dp_gap
                max_x = test if test > max_x else max_x

            # record path
            if max_x > max_y:
                walk[i][j] = '=' if sc_ab >= max_x else 'x'
                score[i][j] = sc_ab if sc_ab >= max_x else max_x
            else:
                walk[i][j] = '=' if sc_ab >= max_y else 'y'
                score[i][j] = sc_ab if sc_ab >= max_y else max_y
            
            # record max 
            if score[i][j] > highest_score:
                highest_score = score[i][j]
                highest_xy = (i,j)
            
    #NEWMAN: NW walk matrix, correction on 1st row and 1st column
    walk[0] = ['y' for j in range(n+1)]
    walk = [ ['x' if j==0 else walk[i][j] for j in range(n+1)] for i in range(m+1)]
                
    #alignment gap insertion 
    ali_short =[]
    ali_long=[]
    while 1:
        if walk[m][n] == '=':
            ali_short.append((m,short_list[m-1]))
            ali_long.append((n,long_list[n-1]))
            m=m-1
            n=n-1
        elif walk[m][n] == 'x':
            ali_short.append((m,short_list[m-1]))
            ali_long.append((-1,'='))
            m=m-1
        else:
            ali_short.append((-1,'='))
            ali_long.append((n,long_list[n-1]))
            n=n-1
        if m==0 and n==0:
            break
            
    long_list = h1_list if isH1Long else h2_list
    ali_result = list(zip(ali_long,ali_short)) if isH1Long else list(zip(ali_short,ali_long))
    ali_result = ali_result[::-1]
    
    #distance measure
    residue_total = 0
    identity_total = 0
    for i,j in ali_result:
        if i==(-1,'=') or j==(-1,'='):
            pass
        else:
            residue_total+=1
            if i==j: 
                identity_total+=1
    
    return identity_total, residue_total, ali_result